The theorems and definitions are with reference to the [accompanying paper](https://arxiv.org/abs/2602.09788).
After installing the package via `pip install qrmfold`, import it.

In [1]:
import qrmfold

# Initializing the Code

Before we construct any gates,
we need to instantiate a quantum Reed-Muller code, $\operatorname{QRM}(m)$ (Definition 4)
parametrized by a positive even number $m$ called the _number of variables_.
This is done with [`qrmfold.QuantumReedMuller`](https://timchan0.github.io/qrmfold/api/qrmfold.quantum_reed_muller.html#qrmfold.quantum_reed_muller.QuantumReedMuller)
passing in $m$ as the `num_variables` parameter.
An example for $m = 4$ is shown below,
which also shows how to
print its stabilizer generators and logical operators.
By default, the constructor minimizes the weight of each stabilizer generator to $2^{m/2 - 1}$
(according to Theorem 2)
but this can be controlled with the `minimize_weight` parameter.

In [2]:
num_variables = 4
code = qrmfold.QuantumReedMuller(num_variables=num_variables, minimize_weight=True)
code.print()

X stabilizer generators:
+X_X_X_X_X_X_X_X_
+_X_X_X_X_X_X_X_X
+__XX__XX__XX__XX
+____XXXX____XXXX
+________XXXXXXXX
Z stabilizer generators:
+Z_Z_Z_Z_Z_Z_Z_Z_
+_Z_Z_Z_Z_Z_Z_Z_Z
+__ZZ__ZZ__ZZ__ZZ
+____ZZZZ____ZZZZ
+________ZZZZZZZZ
logical qubits and their X and Z operators:
0 {1, 2} +___X___X___X___X +____________ZZZZ
1 {1, 3} +_____X_X_____X_X +__________ZZ__ZZ
2 {1, 4} +_________X_X_X_X +______ZZ______ZZ
3 {2, 3} +______XX______XX +_________Z_Z_Z_Z
4 {2, 4} +__________XX__XX +_____Z_Z_____Z_Z
5 {3, 4} +____________XXXX +___Z___Z___Z___Z


The logical qubits are defined by subsets of $[m] := \{1, \dots, m\}$ of cardinality $m/2$.
For example,
the above shows that logical qubit 0 is defined by the subset $\{1, 2\}$.
During instantiation,
we can also specify:
1. the ordering of the logical qubits,
2. the starting logical index.

The default ordering, shown above, is lexicographic with starting index 0,
but the paper defines the canonical ordering (Definition 7) and indexes from 1.
This convention can be done with the `qrmfold.logical_qubit_orderings` submodule as follows:

In [3]:
logical_qubit_ordering = qrmfold.logical_qubit_orderings.canonical(num_variables, start_index=1)
code_canonical = qrmfold.QuantumReedMuller(
    num_variables,
    logical_qubit_ordering=logical_qubit_ordering,
)
code_canonical.print()

X stabilizer generators:
+X_X_X_X_X_X_X_X_
+_X_X_X_X_X_X_X_X
+__XX__XX__XX__XX
+____XXXX____XXXX
+________XXXXXXXX
Z stabilizer generators:
+Z_Z_Z_Z_Z_Z_Z_Z_
+_Z_Z_Z_Z_Z_Z_Z_Z
+__ZZ__ZZ__ZZ__ZZ
+____ZZZZ____ZZZZ
+________ZZZZZZZZ
logical qubits and their X and Z operators:
1 {1, 2} +___X___X___X___X +____________ZZZZ
4 {3, 4} +____________XXXX +___Z___Z___Z___Z
2 {1, 3} +_____X_X_____X_X +__________ZZ__ZZ
5 {2, 4} +__________XX__XX +_____Z_Z_____Z_Z
3 {1, 4} +_________X_X_X_X +______ZZ______ZZ
6 {2, 3} +______XX______XX +_________Z_Z_Z_Z


# Gates from Automorphisms

$P$ and $Q$ are automorphisms $\pi$ of the classical Reed-Muller code (Definition 3).
$\mathsf{U_S}$ and $\mathsf{U_P}$ two types of fold-transversal gates constructed from those automorphisms (Definition 8). Specifically:

The _swap-type fold-transversal gate_ $\mathsf{U_S}(\pi)$ is constructed by
* applying physical $\mathsf{SW}$ to each pair of physical qubits $\{p,\pi(p)\}$ satisfying $p < \pi(p)$, and
* applying nothing ($\mathsf{I}$) to any qubit $q$ satisfying $q = \pi(q)$.

The _phase-type fold-transversal gate_ $\mathsf{U_P}(\pi)$ is constructed by
* applying physical $\mathsf{C_{11}Z}$ to each pair of physical qubits $\{p,\pi(p)\}$ satisfying $p < \pi(p)$, and
* applying physical $\mathsf{S}$ to any qubit $q$ satisfying $q = \pi(q)$.

Let $K=\{(i_1,i_2),\dots,(i_{2c-1},i_{2c})\}$ be a set of $c$ ordered pairs such that $i_1,\dots,i_{2c}\in[m]$ are all distinct.
Define $P(K)$ as the product $P(i_1,i_2)\cdots P(i_{2c-1},i_{2c})$ and similarly for $Q$.

## Physical Circuits

We can get the physical circuits for $\mathsf{U_S}(P(K)), \mathsf{U_P}(P(K)), \mathsf{U_S}(Q(K)), \mathsf{U_P}(Q(K))$ using the `automorphism` method.
The first parameter, `pairs`, is $K$.
Here is an example for $\mathsf{U_S}(P(1, 2) P(3, 4))$:

In [4]:
pairs = [(1, 2), (3, 4)]
physical_circuit = code_canonical.automorphism(pairs, automorphism_type='P', gate_type='swap')
physical_circuit

stim.Circuit('''
    SWAP 9 6 1 2 13 14 10 5 11 7 8 4
''')

The output is in the form of a [`stim.Circuit`](https://github.com/quantumlib/Stim/wiki/Stim-v1.13-Python-API-Reference#stim.Circuit) object,
which can be visualized using its `diagram` method.

In [5]:
physical_circuit.diagram()

q0: ---------------------

 q1: -SWAP----------------
      |
 q2: -SWAP----------------

 q3: ---------------------

 q4: ----------------SWAP-
                     |
 q5: ------SWAP------|----
           |         |
 q6: -SWAP-|---------|----
      |    |         |
 q7: -|----|----SWAP-|----
      |    |    |    |
 q8: -|----|----|----SWAP-
      |    |    |
 q9: -SWAP-|----|---------
           |    |
q10: ------SWAP-|---------
                |
q11: -----------SWAP------

q12: ---------------------

q13: -SWAP----------------
      |
q14: -SWAP----------------

Extending the above,
we can get the physical circuits for $\prod_{L \subseteq K} \mathsf{U}_t(\pi(L))$,
for $t \in \{\mathsf{S}, \mathsf{P}\}$ and $\pi \in \{P, Q\}$,
using the `automorphism_product` method.
Here is an example for $\prod_{L \subseteq \{(1, 2), (3, 4)\}} \mathsf{U_P}(Q(L))$:

In [18]:
pairs = [(1, 2), (3, 4)]
physical_circuit = code_canonical.automorphism_product(pairs, automorphism_type='Q', gate_type='phase')
physical_circuit.diagram()

q0: -S-S-------S-------S-

 q1: -S-S-------S-------S-

 q2: -S-@-------S-@-------
        |         |
 q3: -S-@-------S-@-------

 q4: -S-S-------S-------S-

 q5: -S-S-------S-------S-

 q6: -S-@-------S---@-----
        |           |
 q7: -S-@-------S---@-----

 q8: -S-S---@---------@---
            |         |
 q9: -S-S---|-@-----@-|---
            | |     | |
q10: -S-@---|-|-@-@-|-|---
        |   | | | | | |
q11: -S-@-@-|-|-|-|-|-|-@-
          | | | | | | | |
q12: -S-S-|-@-|-|-|-|-@-|-
          |   | | | |   |
q13: -S-S-|---@-|-|-@---|-
          |     | |     |
q14: -S-@-|-----@-|-----@-
        | |       |
q15: -S-@-@-------@-------

In the above circuit, there are many redundant gates because e.g. $\mathsf{S}^4 =\mathsf{I}$.
A basic depth-reduction technique is account for these repeated S and CZ gates;
this is done by `qrmfold.reduce_circuit_depth`:

In [19]:
qrmfold.reduce_circuit_depth(physical_circuit).diagram()

q0: -------------

 q1: -------------

 q2: -Z-----------

 q3: -Z-----------

 q4: -------------

 q5: -------------

 q6: -Z-----------

 q7: -Z-----------

 q8: -Z-----------

 q9: -Z-----------

q10: -S-@---@-@---
        |   | |
q11: -S-@-@-|-|-@-
          | | | |
q12: -Z---|-|-|-|-
          | | | |
q13: -Z---|-|-|-|-
          | | | |
q14: -S-@-|-@-|-@-
        | |   |
q15: -S-@-@---@---

## Predicted Logical Action

In this section we use theorems from the accompanying paper to predict the logical action
of phase-type gates derived from $Q$ automorphisms.

The logical action of $\mathsf{U_P}(Q(K))$ can be predicted using the following method (which uses Theorem 5):

In [20]:
code_canonical.q_phase_logical_action(pairs)

stim.Circuit('''
    I 1 4 2 5 3 6
    CZ 1 4 2 5 3 6 2 3
    Z 2 3
    CZ 2 6
    Z 2 6
    S 2
''')

...and that of $\prod_{L \subseteq K} \mathsf{U_P}(Q(L))$ can be predicted using the following method (which uses Theorem 6):

In [21]:
code_canonical.q_phase_product_logical_action(pairs)

stim.Circuit('''
    I 1 4 2 5 3 6
    S 2
''')

# Codespace Preservation and Logical Action

Given any unitary physical Clifford circuit $C$ with $2^m$ qubits,
this package can automatically determine:
1. whether $C$ preserves the stabilizer group of $\operatorname{QRM}(m)$,
2. the logical action of $C$ (provided it preserves the stabilizer group).

The first bullet-point is done using the `stabilizers_preserved` method
as shown in the example below,
which determines codespace preservation for the gate
$\prod_{L \subseteq \{(1, 2), (3, 4)\}} \mathsf{U_P}(Q(L))$
(considered in [Physical Circuits](#physical-circuits)).
The `True` indicates that it indeed preserves the codespace.

In [ ]:
code_canonical.stabilizers_preserved(physical_circuit)

True

For the second bullet-point,
the relevant method is `get_logical_tableau`.
We show its usage for the same example as above:

In [11]:
physical_circuit = code_canonical.automorphism_product(pairs, automorphism_type='Q', gate_type='phase')
logical_tableau = code_canonical.get_logical_tableau(physical_circuit=physical_circuit)
print(logical_tableau)

+-xz-xz-xz-xz-xz-xz-
| ++ ++ ++ ++ ++ ++
| XZ __ __ __ __ __
| __ YZ __ __ __ __
| __ __ XZ __ __ __
| __ __ __ XZ __ __
| __ __ __ __ XZ __
| __ __ __ __ __ XZ


What is printed is a [`stim.Tableau`](https://github.com/quantumlib/Stim/wiki/Stim-v1.13-Python-API-Reference#stim.Tableau)
object that specifies how the X and Z operators on each input get transformed.
If this is too cryptic for you,
this class has a method `to_circuit` that synthesizes a circuit that transforms the X and Z operators as specified by the tableau:

In [12]:
logical_tableau.to_circuit().diagram()

q0: -----

q1: -S---

q2: -----

q3: -----

q4: -----

q5: -H-H-

This says that the physical gate induces a logical S on qubit 2
(if we're counting from 1, as was done when we defined `logical_qubit_ordering`
in [Initializing the Code](#initializing-the-code)),
which matches the result in [Predicted Logical Action](#predicted-logical-action).

This workflow allows one to quickly visualize the logical action of
any of the automorphism-derived gates developed in the paper.
Here's another example for $\mathsf{U_S}(P(1, 2))$,
which induces $\overline{\mathsf{SW}}$ on logical qubits $\{2, 6\}$ and $\{3, 5\}$:

In [13]:
physical_circuit = code_canonical.automorphism(pairs=[(1, 2)], automorphism_type='P', gate_type='swap')
code_canonical.get_logical_tableau(physical_circuit).to_circuit().diagram()

q0: -------------

q1: -X-@-X-------
     | | |
q2: -|-|-|-X-@-X-
     | | | | | |
q3: -|-|-|-|-|-|-
     | | | | | |
q4: -|-|-|-@-X-@-
     | | |
q5: -@-X-@-------

# Addressable Gates

An $\overline{\mathsf{S}}$, $\overline{\mathsf{H}}$, $\overline{\mathsf{C_{00}Z}}$, or $\overline{\mathsf{SW}}$ gate
on any (pair of) logical qubit(s) can be obtained using the `gate` method,
which outputs the physical circuit that induces the logical gate.
By default, this method applies the basic depth reduction from `qrmfold.reduce_circuit_depth`.
The `name` parameter can be 'S', 'H', 'ZZCZ', or 'SWAP'.
The `targets` parameter behaves similarly to `stim.Circuit.append`
in that if the gate is a 2-qubit gate,
the operation will be broadcasted over consecutive target pairs.
E.g. a logical Hadamard on qubit 1 looks like:

In [14]:
code_canonical.gate(name='H', targets=[1]).diagram()

q0: -----------H-------------H---------------

 q1: -----------H---Z---------H---------------

 q2: -----------H---Z---------H---------------

 q3: -----------H-S---@---@-@-H---------------
                      |   | |
 q4: -Z---------H-----|---|-|-H---Z-----------
                      |   | |
 q5: -Z---------H---Z-|---|-|-H---Z-----------
                      |   | |
 q6: -Z---------H---Z-|---|-|-H---Z-----------
                      |   | |
 q7: -Z---------H---S-@-@-|-|-@-H-Z-----------
                        | | | |
 q8: -Z---------H-------|-|-|-|-H-Z-----------
                        | | | |
 q9: -Z---------H---Z---|-|-|-|-H-Z-----------
                        | | | |
q10: -Z---------H---Z---|-|-|-|-H-Z-----------
                        | | | |
q11: -Z---------H-S---@-|-@-|-@-H-Z-----------
                      | |   |
q12: -S-@-@---@-H-----|-|---|---H-S-@-@---@---
        | |   |       | |   |       | |   |
q13: -S-@-|-@-|-@-H-Z-|-|---|---H-S-@-|-@-|-@-
          | | | |     | |   |         | | | |
q14: -S-@-@-|-|-@-H-Z-|-|---|---H-S-@-@-|-|-@-
        |   | |       | |   |       |   | |
q15: -S-@---@-@---H-S-@-@---@---H-S-@---@-@---

Another e.g.; one $\overline{\mathsf{C_{00}Z}}$ gate on qubits 1 and 3
and one $\overline{\mathsf{C_{00}Z}}$ gate on qubits 2 and 4
looks like:

In [15]:
code_canonical.gate(name='ZZCZ', targets=[1, 3, 2, 4]).diagram()

q0: ---------------

 q1: ---------------

 q2: -S_DAG-------@-
                  |
 q3: -S_DAG-----@-|-
                | |
 q4: -S_DAG-@---|-|-
            |   | |
 q5: -S_DAG-@---|-|-
                | |
 q6: -Z-----@---|-@-
            |   |
 q7: -Z-----@---@---

 q8: ---------------

 q9: ---------------

q10: -S_DAG-----@---
                |
q11: -S_DAG---@-|---
              | |
q12: -S_DAG-@-|-|---
            | | |
q13: -S_DAG-@-|-|---
              | |
q14: -Z-----@-|-@---
            | |
q15: -Z-----@-@-----

Note the circuit depth of a 2-qubit gate depends on how different the subsets defining the logical qubits are.
For example, one $\overline{\mathsf{C_{00}Z}}$ gate on qubits 1 and 4 is quite deep:

In [16]:
code_canonical.gate('ZZCZ', [1, 4])

stim.Circuit('''
    REPEAT 3 {
        Z 0 1 2 3 12 13 14 15
        S_DAG 4 5 6 7
        CZ 12 14 13 15 4 5 6 7 12 15 13 14
        H 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
        S 11 3 15 7
        Z 1 2 5 6 9 10 13 14
        CZ 3 7 11 15 7 15 3 11 3 15 7 11
        H 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
        Z 2 3 8 9 14 15
        S_DAG 12 13 6 7
        CZ 12 13 6 7 12 14 13 15 12 15 13 14 7 15 6 14 6 15 7 14
        H 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
        S 9 11 13 15
        Z 1 3 5 7 8 10 12 14
        CZ 9 11 13 15 11 15 9 13 11 13 9 15
        H 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
        S 14 15 6 7
        Z 2 3 4 5 10 11 12 13
        CZ 6 7 14 15 7 15 6 14 6 15 7 14
    }
    S 2 3 6 7 10 11 14 15
    Z 0 1 4 5 8 9 12 13
    CZ 7 15 3 11 2 10 6 14
    REPEAT 3 {
        Z 2 3 4 5 10 11 12 13
        S_DAG 7 6 14 15
        CZ 7 14 6 15 6 14 7 15 14 15 6 7
        H 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 0
        Z 1 3 5 7 8 10 12 14
        S_DAG 9 11 13 1